In [2]:
import folium
import pandas as pd
from folium.plugins import HeatMap
from folium import Icon
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import os

In [3]:
df = pd.read_csv('../results/CreateLabellingResult.csv')

In [4]:
df

,timestamp,username,handle,tweet_date,content,content_ment_link,content_wo_punct,content_wo_removed_english,content_wo_normalize,content_wo_tokenize,...,location_unique,tokenized_location,address,latitude,longitude,city,latitude_city,longitude_city,initial_label,final_label
0,2024-11-03 19:38:25,Orhan Şevik,orhan_sevik,"Feb 8, 2023",@haluklevent\n @oguzhanugur\n hepinizden alla...,\n \n hepinizden allah razı olsun iyiki varsı...,hepinizden allah razı olsun iyiki varsını...,hepinizden allah razı olsun iyiki varsınız deprem,hepinizden allah razı olsun iyi ki varsınız de...,"['hepinizden', 'allah', 'razı', 'olsun', 'iyi'...",...,NaN,[],NaN,NaN,NaN,[],NaN,NaN,destek bilgilendirme,destek
1,2024-11-03 19:38:25,nursena,nurssxx_,"Feb 8, 2023",allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver deprem,allahım nolur onlara dayanma gücü ver deprem,allahım ne olur onlara dayanma gücü ver deprem,"['allahım', 'ne', 'olur', 'onlara', 'dayanma',...",...,NaN,[],NaN,NaN,NaN,[],NaN,NaN,destek bilgilendirme,destek
2,2024-11-03 19:38:25,Elif ŞEKER,55ellllllif,"Feb 8, 2023",replying to \n@haluklevent\n @danlabilic\n and...,replying to \n\n \n and \n_harun\nadıyamanda a...,replying to and harun adıyamanda ali taş...,harun adıyamanda ali taşı mahallesi sokak hic...,harun adıyamanda ali taşı mahallesi sokak hicr...,"['harun', 'adıyamanda', 'ali', 'taşı', 'mahall...",...,Adıyaman Ali Taşı Mahallesi Sokak Hicret Apart...,"['Adıyaman', 'Ali', 'Taşı', 'Mahallesi', 'Soka...",NaN,NaN,NaN,['Adıyaman'],37.789360,38.314110,destek bilgilendirme,destek
3,2024-11-03 19:38:25,Yusuf,yusufaltuns,"Feb 8, 2023",tuğba södekoğlu kovulsun \n@showtv\n #deprem,tuğba södekoğlu kovulsun \n\n #deprem,tuğba södekoğlu kovulsun deprem,tuğba södekoğlu kovulsun deprem,tuğba sodekoğlu kovulsun deprem,"['tuğba', 'sodekoğlu', 'kovulsun', 'deprem']",...,NaN,[],NaN,NaN,NaN,[],NaN,NaN,destek bilgilendirme,destek
4,2024-11-03 19:38:25,𝐎̈𝐳𝐠𝐮̈𝐫 𝐑𝐚𝐧,dryghtn2,"Feb 8, 2023",arkadaşimiza ulaşamiyoruz\nkahramanmaraş elbis...,arkadaşimiza ulaşamiyoruz\nkahramanmaraş elbis...,arkadaşimiza ulaşamiyoruz kahramanmaraş elbist...,arkadaşimiza ulaşamiyoruz kahramanmaraş elbist...,arkadaşımıza ulaşamıyoruz kahramanmaraş elbist...,"['arkadaşımıza', 'ulaşamıyoruz', 'kahramanmara...",...,Kahramanmaraş Elbistan Pınarbaşı Mahallesi Cad...,"['Kahramanmaraş', 'Elbistan', 'Pınarbaşı', 'Ma...",kahramanmaraş,NaN,NaN,['Kahramanmaraş'],37.783034,36.830655,çok acil acil,acil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2097,2024-11-17 16:28:31,KEREMCGRC,keremcgrci,"Feb 8, 2023",yardım ağı´na destek ol! \n\ntwitter'da paylaş...,yardım ağı´na destek ol! \n\ntwitter'da paylaş...,yardım ağı´na destek ol twitterda paylaşılan...,yardım ağı´na destek ol twitterda paylaşılan a...,yardım ağına destek ol twitterda paylaşılan ad...,"['yardım', 'ağına', 'destek', 'ol', 'twitterda...",...,Hataykahramanmaraşhatay,['Hataykahramanmaraşhatay'],NaN,NaN,NaN,[],NaN,NaN,destek bilgilendirme,destek
2098,2024-11-17 16:28:31,aMMo,tatavaypmadvmet,"Feb 8, 2023",teyi̇tli̇ lütfen yardim\nemlak bank evleri 1. ...,teyi̇tli̇ lütfen yardim\nemlak bank evleri 1. ...,teyi̇tli̇ lütfen yardim emlak bank evleri kıs...,teyi̇tli̇ lütfen yardim emlak evleri kısım blo...,teyi̇tli̇ lütfen yardım emlak evleri kısım blo...,"['teyi̇tli̇', 'lütfen', 'yardım', 'emlak', 'ev...",...,Hatay Antakya Cebrail Mahallesi,"['Hatay', 'Antakya', 'Cebrail', 'Mahallesi']",hatay antakya,36.202547,36.160291,['Hatay'],36.345133,36.074802,çok acil acil,çok acil
2099,2024-11-17 16:28:31,N’Y,nyonyedi,"Feb 8, 2023",05324707903 babalatv\n\nwhastapp hatti tüm böl...,05324707903 babalatv\n\nwhastapp hatti tüm böl...,babalatv whastapp hatti tüm bölgelerdeki yar...,babalatv whastapp hatti tüm bölgelerdeki yardi...,babalatv Whatsapp hattı tüm bölgelerdeki yardı...,"['babalatv', 'Whatsapp', 'hattı', 'tüm', 'bölg...",...,NaN,[],NaN,NaN,NaN,[],NaN,NaN,çok acil 

In [20]:
label_styles = {
    "çok acil": {"color": "red", "icon": "exclamation-circle"},
    "bilgilendirme": {"color": "blue", "icon": "info-circle"},
    "destek": {"color": "green", "icon": "question-circle"},
    "acil": {"color": "orange", "icon": "circle"}
}

In [ ]:
m = folium.Map(location=[37.3741, 36.7784], zoom_start=7)

for index, row in df.iterrows():
    if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
        label = row["final_label"]
        if label in label_styles:
            marker_color = label_styles[label]["color"]
            marker_icon = label_styles[label]["icon"]
        else:
            marker_color = "gray"  
            marker_icon = "circle"

        tweet_content = row["content"]  
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=f'<strong>Tweet Content:</strong><br>{tweet_content}',  
            icon=Icon(color=marker_color, icon=marker_icon)
        ).add_to(m)
m.save("../web/address_map_new.html")

# HEATMAP

In [4]:
folder_path = "../ParcaliSonuc"

# Klasördeki tüm dosyaları listeleyip gerekli CSV dosyalarını seçiyoruz
csv_files = [file for file in os.listdir(folder_path) if file.startswith('df') and file.endswith('coordinatecity.csv')]

# Boş bir liste oluşturuyoruz ve her bir CSV'yi okuyup ekliyoruz
dataframes = []
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Tüm DataFrame'leri birleştiriyoruz
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df

,Date,Tweet,lower,content_ment_link,content_wo_punct,content_wo_removed_english,content_wo_normalize,content_wo_tokenize,content_wo_stop,content_wo_lemmatized,content_no_rare_words,content_wo_tokenize_capitalize,location,location_unique,tokenized_location,city,latitude_city,longitude_city
0,08.02.2023,@haluklevent\n @oguzhanugur\n hepinizden alla...,@haluklevent\n @oguzhanugur\n hepinizden alla...,\n \n hepinizden allah razı olsun iyiki varsı...,hepinizden allah razı olsun iyiki varsını...,hepinizden allah razı olsun iyiki varsınız deprem,hepinizden allah razı olsun iyi ki varsınız de...,"['hepinizden', 'allah', 'razı', 'olsun', 'iyi'...","['hepinizden', 'razı', 'varsınız', 'deprem']","['hep', 'razı', 'var', 'deprem']","['hep', 'razı', 'var', 'deprem']",Hepinizden Allah Razı Olsun Iyi Ki Varsınız De...,NaN,NaN,[],[],NaN,NaN
1,08.02.2023,allah ın cezası.sen nasıl bir vicdansız mahlu...,allah ın cezası.sen nasıl bir vicdansız mahlu...,allah ın cezası.sen nasıl bir vicdansız mahlu...,allah ın cezasısen nasıl bir vicdansız mahluk...,allah ın cezasısen nasıl bir vicdansız mahluka...,allahın cezasisen nasıl bir vicdansız mahlukat...,"['allahın', 'cezasisen', 'nasıl', 'bir', 'vicd...","['allahın', 'cezasisen', 'vicdansız', 'mahluka...","['allah', 'cezasisen', 'vicdan', 'mahlukat', '...","['allah', 'cezasisen', 'vicdan', 'mahlukat', '...",Allahın Cezasisen Nasıl Bir Vicdansız Mahlukat...,Maraş Hatay,Maraş Hatay,"['Maraş', 'Hatay']",['Hatay'],36.345133,36.074802
2,08.02.2023,koskaca bir aile enkaz altındaymış\n\nkucukdal...,koskaca bir aile enkaz altındaymış\n\nkucukdal...,koskaca bir aile enkaz altındaymış\n\nkucukdal...,koskaca bir aile enkaz altındaymış kucukdalya...,koskaca bir enkaz altındaymış kucukdalyan maha...,koskaca bir enkaz altındaymış küçükdalyan maha...,"['koskaca', 'bir', 'enkaz', 'altındaymış', 'kü...","['koskaca', 'enkaz', 'altındaymış', 'küçükdaly...","['koska', 'enkaz', 'alt', 'küçükdalyan', 'maha...","['koska', 'enkaz', 'alt', 'küçükdalyan', 'maha...",Koskaca Bir Enkaz Altındaymış Küçükdalyan Maha...,Küçükdalyan Mahallesi Emir Sokak Subaşı Apartmanı,Küçükdalyan Mahallesi Emir Sokak Subaşı Apartmanı,"['Küçükdalyan', 'Mahallesi', 'Emir', 'Sokak', ...",[],NaN,NaN
3,08.02.2023,ülkeyi yıkıp gidecekler dedikleri buymuş! yıkt...,ülkeyi yıkıp gidecekler dedikleri buymuş! yıkt...,ülkeyi yıkıp gidecekler dedikleri buymuş! yıkt...,ülkeyi yıkıp gidecekler dedikleri buymuş yıktı...,ülkeyi yıkıp gidecekler dedikleri buymuş yıktı...,ülkeyi yıkıp gidecekler dedikleri buymuş yıktı...,"['ülkeyi', 'yıkıp', 'gidecekler', 'dedikleri',...","['ülkeyi', 'yıkıp', 'gidecekler', 'dedikleri',...","['ülke', 'yık', 'gid', 'de', 'buy', 'yık', 'ül...","['ülke', 'yık', 'gid', 'de', 'buy', 'yık', 'ül...",Ülkeyi Yıkıp Gidecekler Dedikleri Buymuş Yıktı...,NaN,NaN,[],[],NaN,NaN
4,07.02.2023,önemli̇ duyuru \n@depremdairesi\n @afadbaskanl...,önemli̇ duyuru \n@depremdairesi\n @afadbaskanl...,önemli̇ duyuru \n\n \n _dernegi\n \n \n #adiya...,önemli̇ duyuru dernegi adiyaman depr...,önemli̇ duyuru dernegi adiyaman deprem hatayya...,önemli duyuru derneği adıyaman deprem hatayyar...,"['önemli', 'duyuru', 'derneği', 'adıyaman', 'd...","['önemli', 'duyuru', 'derneği', 'adıyaman', 'd...","['önem', 'duyuru', 'derneğ', 'adıyaman', 'depr...","['önem', 'duyuru', 'derneğ', 'adıyaman', 'depr...",Önemli Duyuru Derneği Adıyaman Deprem Hatayyar...,Adıyaman,Adıyaman,['Adıyaman'],['Adıyaman'],37.789360,38.314110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844815,10.02.2023,"i̇nandiğiniz di̇n bu deği̇l. okuyun, araştirin...","i̇nandiğiniz di̇n bu deği̇l. okuyun, araştirin...","i̇nandiğiniz di̇n bu deği̇l. okuyun, araştirin...",i̇nandiğiniz di̇n bu deği̇l okuyun araştirin ö...,i̇nandiğiniz di̇n deği̇l okuyun araştirin öğre...,inandığınız din değil okuyun araştırın öğrenin...,"['inandığınız', 'din', 'değil', 'okuyun', 'ara...","['inandığınız', 'din', 'okuyun', 'araştırın', ...","['inan', 'din', 'oku', 'araştır', 'öğren', 'af...","['inan', 'din'

In [5]:
geolocator = Nominatim(user_agent="geoapi")
def get_coordinates(address):
    if not address or pd.isna(address):
        return None, None
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None

In [6]:
non_empty_cities = combined_df['city'].dropna()
non_empty_cities = non_empty_cities[non_empty_cities.str.strip() != '']
city_exploded = non_empty_cities.str.replace(r"[\[\]']+", '', regex=True)  
city_exploded = city_exploded.str.split(',').explode().str.strip()
city_count = city_exploded.value_counts()
city_count_df = city_count.reset_index()
city_count_df.columns = ['City', 'Count']
city_count_df = city_count_df[city_count_df['City'] != '']
city_count_df.head(20)

,City,Count
1,Hatay,64711
2,Kahramanmaraş,41372
3,Adıyaman,35419
4,Malatya,15843
5,Gaziantep,14564
6,Adana,9617
7,Diyarbakır,8542
8,İstanbul,7488
9,Ankara,4839
10,Şanlıurfa,4508


In [7]:
city_count_df[['Latitude', 'Longitude']] = city_count_df['City'].apply(lambda x: pd.Series(get_coordinates(x)))

In [8]:
city_count_df

,City,Count,Latitude,Longitude
1,Hatay,64711,36.345133,36.074802
2,Kahramanmaraş,41372,37.783034,36.830655
3,Adıyaman,35419,37.789360,38.314110
4,Malatya,15843,38.348715,38.319067
5,Gaziantep,14564,37.062832,37.379262
...,...,...,...,...
77,Karabük,34,41.111035,32.619390
78,Burdur,32,37.518341,30.169125
79,Kırklareli,30,41.707805,27.605133
80,Çankırı,30,40.666769,33.452607


In [10]:
m = folium.Map(location=[41.008238, 28.978359], zoom_start=6)
heat_data = [[row['Latitude'], row['Longitude'], row['Count']] for index, row in city_count_df.iterrows()]
HeatMap(heat_data, 
        min_opacity=0.4,  
        max_opacity=0.9, 
        radius=50,  
        blur=30,  
        gradient={  
            0.2: 'blue',   
            0.5: 'lime',   
            0.7: 'yellow',  
            0.8: 'orange',
            1.0: 'red'      
        }).add_to(m)

for index, row in city_count_df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=10,  
        color='black',
        fill=True,
        fill_opacity=0.7,
        popup=f"{row['City']}: {row['Count']}",
        tooltip=f"{row['City']} - {row['Count']} people"
    ).add_to(m)

m.save('../web/heatmap.html')